In [69]:
import pandas as pd

#Load files
X_full = pd.read_csv("/home/manu/Documents/Data Science/Projects/Titanic/train.csv", index_col = "PassengerId")
X_test_full = pd.read_csv("/home/manu/Documents/Data Science/Projects/Titanic/test.csv", index_col = "PassengerId")

In [2]:
#Explore data

#X_full.head()
#X_full.shape
#X_full.isnull().sum()
#X_full.describe
#list ((X_full.dtypes == 'object')[X_full.dtypes == 'object'].index)
#X_full['Sex'].unique()
#set(X_full['Sex'])
#X_full['Sex'].nunique()
#X_full.info()
#X_full['Survived'].value_counts(normalize=True)
#X_full['Survived'].groupby(X_full['Pclass']).mean()
#pd.qcut(X_full['Fare'], 3).value_counts()
#X_full['Survived'].groupby(pd.qcut(X_full['Fare'], 3)).mean()
#X_full['Survived'].groupby(X_full['Sex']).sum()
#pd.crosstab(pd.qcut(X_full['Fare'], 5), columns=X_full['Pclass'])
#X_full['Survived'].corr(X_full['Fare'])

In [72]:
from sklearn.model_selection import train_test_split

#Remove rows with missing target
X_full.dropna(axis=0, subset = ['Survived'], inplace=True)

#Separate target from predictors
y = X_full.Survived
X_full.drop(['Survived'], axis=1, inplace = True)


In [73]:
#Select object (Only low cardinality ones) and numerical columns

object_cols = [col for col in X_full.columns if (X_full[col].dtype == 'object') and X_full[col].nunique() < 10]

num_cols = [col for col in X_full.columns if X_full[col].dtype in ['int','float64']]

my_cols = object_cols + num_cols

X = X_full[my_cols].copy()
X_test = X_test_full [my_cols].copy()

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

#Impute categorical missing values with most frequent value and numerical with median
my_categ_imputer = SimpleImputer(strategy = 'most_frequent')
my_num_imputer = SimpleImputer (strategy = 'median')

X[object_cols] = my_categ_imputer.fit_transform(X[object_cols])
X_test[object_cols] = my_categ_imputer.fit_transform(X_test[object_cols])
X[num_cols] = my_num_imputer.fit_transform(X[num_cols])
X_test[num_cols] = my_num_imputer.fit_transform (X_test[num_cols])

#Label encode categorical variables
label_encoder = LabelEncoder()
for col in object_cols:
    X[col] = label_encoder.fit_transform (X[col])
    X_test[col] = label_encoder.transform (X_test[col])

In [14]:
from sklearn.ensemble import RandomForestClassifier

#Train model
#My_model = RandomForestRegressor(random_state=1)
#My_model = RandomForestRegressor(n_estimators=50, random_state=0)
My_model = RandomForestClassifier(n_estimators=3, random_state=0)
#My_model = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
#My_model = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
#My_model = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

#My_model.fit(X_train, y_train)



In [15]:
from sklearn.model_selection import cross_val_score

#Messure model performance
scores = cross_val_score(My_model,X,y,cv=5, scoring = 'accuracy')

print ("Average accuracy score (across experiments): ",scores.mean())


Average accuracy score (across experiments):  0.793528340970435


In [16]:
#Best result so far:
#My_model = RandomForestRegressor(random_state=1)
#-> 81%


In [17]:
#fit model and get predictions

My_model.fit(X, y)

#Get prediction for test
Test_prediction = My_model.predict(X_test)

#Prepare data to be saved in a CSV

Test_prediction = pd.DataFrame({'Survived':Test_prediction}, index = X_test.index)

#save data to CSV 
Test_prediction.to_csv ("/home/manu/Documents/Data Science/Projects/Titanic/result_RF_NoPipeline.csv")

